# 1) Extract map feature names

In [1]:
from bs4 import BeautifulSoup

In [2]:
from urllib.request import urlopen
html = urlopen("https://wiki.openstreetmap.org/wiki/Map_features").read()

In [3]:
soup = BeautifulSoup(html, 'html.parser')

In [4]:
tags=[]
for tag in soup.find_all('a'):
    tags.append(tag.get('title'))

In [5]:
'library (page does not exist)'.replace(' (page does not exist)','')

'library'

In [6]:
tags=[tag for tag in tags if tag]
tags=[tag for tag in tags if 'Tag' in tag]
tags=[tag for tag in tags if '=' in tag]
tags=[tag[4:] for tag in tags]
tags=[tag.replace(" (page does not exist)","") for tag in tags]

tags=list(set(tags))
tags=[tag.split("=") for tag in tags]

In [7]:
tags

[['railway', 'tram'],
 ['amenity', 'arts centre'],
 ['route', 'inline skates'],
 ['direction', 'clockwise'],
 ['shop', 'video'],
 ['cycleway', 'opposite share busway'],
 ['shop', 'electronics'],
 ['shop', 'kitchen'],
 ['amenity', 'public building'],
 ['building', 'kiosk'],
 ['route', 'ski'],
 ['amenity', 'waste transfer station'],
 ['landuse', 'salt pond'],
 ['shop', 'chemist'],
 ['shop', 'ticket'],
 ['building', 'gatehouse'],
 ['building', 'sty'],
 ['route', 'canoe'],
 ['landuse', 'winter sports'],
 ['highway', 'trailhead'],
 ['building', 'retail'],
 ['highway', 'unclassified'],
 ['shop', 'brewing supplies'],
 ['cycleway', 'share busway'],
 ['highway', 'road'],
 ['building', 'ger'],
 ['shop', 'doityourself'],
 ['landuse', 'residential'],
 ['amenity', 'parking space'],
 ['shop', 'perfumery'],
 ['shop', 'water'],
 ['building', 'pavilion'],
 ['healthcare', 'nutrition counselling'],
 ['landuse', 'orchard'],
 ['building', 'chapel'],
 ['shop', 'tobacco'],
 ['shop', 'furniture'],
 ['shop', '

In [8]:
tag_dict=dict()
key_set=set([tag[0] for tag in tags])
val_list=[tag[1] for tag in tags]
for key in key_set:
    value=[tag[1] for tag in tags if tag[0]==key]
    tag_dict[key]=value

In [9]:
tag_dict['tourism']=['hotel','attraction','artwork','museum']
tag_dict['amenity']

['arts centre',
 'public building',
 'waste transfer station',
 'parking space',
 'studio',
 'car rental',
 'planetarium',
 'post box',
 'baking oven',
 'college',
 'conference centre',
 'ranger station',
 'fountain',
 'embassy',
 'brothel',
 'photo booth',
 'swingerclub',
 'baby hatch',
 'parking entrance',
 'community centre',
 'boat rental',
 'parking',
 'social centre',
 'townhall',
 'public bookcase',
 'grave yard',
 'food court',
 'charging station',
 'gym',
 'bicycle parking',
 'vending machine',
 'library',
 'fire station',
 'crematorium',
 'motorcycle parking',
 'bureau de change',
 'social facility',
 'kneipp water cure',
 'bicycle repair station',
 'cinema',
 'fuel',
 'place of worship',
 'restaurant',
 'university',
 'love hotel',
 'nursing home',
 'funeral hall',
 'music school',
 'toy library',
 'driving school',
 'bar',
 'bbq',
 'telephone',
 'gambling',
 'police',
 'recycling',
 'doctors',
 'theatre',
 'post office',
 'toilets',
 'bank',
 'animal shelter',
 'school',
 '

In [10]:
to_remove=["cutting","aeroway","fuel","oven","power","access","type","horse","military","oneway","route","highway"]

In [11]:
#for key in to_remove:
#    del tag_dict[key]

In [12]:
tag_dict.keys()

dict_keys(['tunnel', 'power', 'social facility', 'footway', 'bridge', 'shop', 'highway', 'boundary', 'access', 'oven', 'healthcare', 'junction', 'natural', 'landuse', 'recycling type', 'aeroway', 'substation', 'healthcare:speciality', 'service', 'industrial', 'water', 'public transport', 'railway', 'man made', 'barrier', 'type', 'place', 'busway', 'leisure', 'historic', 'tourism', 'cycleway', 'office', 'building', 'military', 'fuel', 'emergency', 'cutting', 'oneway', 'craft', 'site', 'direction', 'residential', 'bicycle', 'amenity', 'route'])

In [13]:
#tag_dict={'tourism':['hotel', 'attraction', 'artwork', 'museum'], 'amenity':['fast_food','arts_centre']}

In [14]:
tag_dict

{'tunnel': ['yes'],
 'power': ['substation'],
 'social facility': ['nursing home'],
 'footway': ['sidewalk', 'crossing'],
 'bridge': ['yes'],
 'shop': ['video',
  'electronics',
  'kitchen',
  'chemist',
  'ticket',
  'brewing supplies',
  'doityourself',
  'perfumery',
  'water',
  'tobacco',
  'furniture',
  'toys',
  'games',
  'frozen food',
  'household linen',
  'dry cleaning',
  'fashion',
  'department store',
  'appliance',
  'greengrocer',
  'trade',
  'collector',
  'tailor',
  'window blind',
  'car',
  'ice cream',
  'golf',
  'video games',
  'tyres',
  'computer',
  'caravan',
  'bicycle',
  'model',
  'variety store',
  'cannabis',
  'doors',
  'vacuum cleaner',
  'wholesale',
  'farm',
  'butcher',
  'organic',
  'pastry',
  'deli',
  'paint',
  'wine',
  'cosmetics',
  'carpet',
  'lighting',
  'atv',
  'copyshop',
  'chocolate',
  'spices',
  'outdoor',
  'erotic',
  'gas',
  'bookmaker',
  'photo',
  'security',
  'religion',
  'florist',
  'swimming pool',
  'secon

We will try the code on only "shop" and "amenity"

In [15]:
#tag_dict=dict((k, tag_dict[k]) for k in ('amenity', 'shop'))

In [16]:
#print(tag_dict, file=open("tag_dict.txt", "a"))

# 2) Downloading metadata using Overpass API by OpenStreetMap

In [1]:
from OSMPythonTools.overpass import Overpass
overpass = Overpass()

In [1]:
def divide_region(bbox, axis_divisions=4):
    
    min_lat=bbox[0]
    min_long=bbox[1]
    max_lat=bbox[2]
    max_long=bbox[3]
    
    dist_along_lat=max_lat-min_lat
    dist_along_long=max_long-min_long
    
    return [(min_lat+dist_along_lat*(i-1)/4,min_long+dist_along_long*(j-1)/4, min_lat+dist_along_lat*i/4, min_long+dist_along_long*j/4) for j in range(1,axis_divisions+1) for i in range(1,axis_divisions+1)]

In [3]:
bbox=[42.189829808246266, -71.2912398492822, 42.48199986253678, -70.91697706096569] #Boston (from network)
# bbox=[37.52092319,-122.55478906,37.92694855,-122.27040728] #San Francisco (from network)
# bbox=[35.45, 10.994901048123777, 35.55165622800387, 11.09725485069693] #Mahdia

In [1]:
# divide_region(bbox)

In [20]:
# import pickle
# import numpy as np
# with open('graph_clusters_30m_biasCorr_sf.pk','rb') as file:
#     graph=pickle.load(file,encoding='latin1')
# np.array(graph.vs['pos']).min(axis=0)

In [20]:
import folium

regions=divide_region(bbox, 4)

mapit = folium.Map( location=[0.5*bbox[0]+0.5*bbox[2], 0.5*bbox[1]+0.5*bbox[3]], zoom_start=10 )


folium.Marker( location=[ bbox[0],bbox[1] ], radius=8).add_to( mapit)
folium.Marker( location=[ bbox[0],bbox[3] ], radius=8).add_to( mapit)
folium.Marker( location=[ bbox[2],bbox[1] ], radius=8).add_to( mapit)
folium.Marker( location=[ bbox[2],bbox[3] ], radius=8).add_to( mapit)

    
mapit

In [22]:
mapit.save('map.html')

### Download attributes

In [23]:
map_features=[]
for region in divide_region(bbox, 4):
    print(divide_region(bbox, 4).index(region))
    south=region[0]
    west=region[1]
    north=region[2]
    east=region[3]
    print(south, west, north, east)
    for key in list(tag_dict.keys()):
        for value in tag_dict[key]:
            query="""
        
    node["""+'"'+key+'"'+"""="""+'"'+value+'"'+"""]("""+str(south)+','+str(west)+','+str(north)+','+str(east)+""");
    out body;
    """
            result = overpass.query(query)
        
            for elt in result.elements():
                feature=[key, value]
                if elt.lat() and elt.lon():
                    feature.append(elt.lat())
                    feature.append(elt.lon())
                    feature.append(elt.id())
                    print(feature)
                    print(divide_region(bbox, 4).index(region))
                    map_features.append(feature)
                    



[overpass] downloading data: [timeout:25][out:json];
        
    node["amenity"="baking oven"](35.45,10.994901048123777,35.47541405700097,11.020489498767066);
    out body;
    


0
35.45 10.994901048123777 35.47541405700097 11.020489498767066


[overpass] downloading data: [timeout:25][out:json];
        
    node["amenity"="funeral hall"](35.45,10.994901048123777,35.47541405700097,11.020489498767066);
    out body;
    


KeyboardInterrupt: 

In [36]:
### Adding what's left

for key in list(tag_dict.keys()):
    for value in tag_dict[key]:
        query="""
        
    node["""+'"'+key+'"'+"""="""+'"'+value+'"'+"""]("""+str(bbox[0])+','+str(bbox[1])+','+str(bbox[2])+','+str(bbox[3])+""");
    out body;
    """
        result = overpass.query(query)
        
        for elt in result.elements():
            feature=[key, value]
            if elt.lat() and elt.lon():
                feature.append(elt.lat())
                feature.append(elt.lon())
                feature.append(elt.id())
                print(feature)
                map_features.append(feature)
                    

['amenity', 'library', 35.506923, 11.0500038, 1935939159]
['amenity', 'atm', 35.5400087, 11.0312101, 558051370]
['amenity', 'atm', 35.5121105, 11.0466244, 1935883793]
['amenity', 'restaurant', 35.5273041, 11.0381655, 558051342]
['amenity', 'restaurant', 35.5306997, 11.036767, 558051358]
['amenity', 'restaurant', 35.5057405, 11.0642451, 558059992]
['amenity', 'restaurant', 35.5035656, 11.0686636, 3416128780]
['amenity', 'restaurant', 35.5021919, 11.0666816, 4816660723]
['amenity', 'restaurant', 35.5022834, 11.0668669, 4816677226]
['amenity', 'restaurant', 35.5044387, 11.0682815, 6013862332]
['amenity', 'restaurant', 35.4783795, 11.0530179, 6284535865]
['amenity', 'restaurant', 35.5038901, 11.0708308, 6379307839]
['amenity', 'restaurant', 35.5044592, 11.0724362, 6379307840]
['amenity', 'restaurant', 35.5045115, 11.0735326, 6379307844]
['amenity', 'restaurant', 35.5030544, 11.069397, 6433933436]
['amenity', 'restaurant', 35.5049993, 11.066824, 6537591255]
['amenity', 'restaurant', 35.5050

[overpass] downloading data: [timeout:25][out:json];
        
    node["railway"="tram stop"](35.45,10.994901048123777,35.55165622800387,11.09725485069693);
    out body;
    


['shop', 'massage', 35.5437307, 11.0305732, 6649563006]
['shop', 'seafood', 35.5029665, 11.0697019, 1935966222]
['shop', 'gift', 35.5435714, 11.0301917, 6649563010]
['shop', 'gift', 35.5412178, 11.0286732, 6784644969]
['shop', 'computer', 35.5070271, 11.0493048, 4791040522]
['shop', 'florist', 35.5220722, 11.0369775, 4608083491]
['shop', 'doityourself', 35.5132067, 11.0444516, 1935889090]
['shop', 'butcher', 35.5029272, 11.0699594, 1935966529]
['shop', 'clothes', 35.5122854, 11.0468522, 1935897293]
['shop', 'clothes', 35.5035451, 11.0691334, 1935971250]
['shop', 'clothes', 35.5038683, 11.0678244, 1935979665]
['shop', 'clothes', 35.5032962, 11.0672745, 1935980421]
['shop', 'clothes', 35.5077863, 11.0562988, 4009732249]
['shop', 'clothes', 35.511192, 11.052785, 5765671453]
['shop', 'clothes', 35.5077571, 11.0564052, 5842719387]
['shop', 'clothes', 35.5075957, 11.0563947, 5842719388]
['railway', 'station', 35.4999902, 11.0481943, 5227817817]
['railway', 'station', 35.5061741, 11.0302648, 

[overpass] downloading data: [timeout:25][out:json];
        
    node["highway"="toll gantry"](35.45,10.994901048123777,35.55165622800387,11.09725485069693);
    out body;
    
[overpass] downloading data: [timeout:25][out:json];
        
    node["highway"="footway"](35.45,10.994901048123777,35.55165622800387,11.09725485069693);
    out body;
    
[overpass] downloading data: [timeout:25][out:json];
        
    node["highway"="trunk link"](35.45,10.994901048123777,35.55165622800387,11.09725485069693);
    out body;
    
[overpass] downloading data: [timeout:25][out:json];
        
    node["highway"="tertiary link"](35.45,10.994901048123777,35.55165622800387,11.09725485069693);
    out body;
    
[overpass] downloading data: [timeout:25][out:json];
        
    node["highway"="bridleway"](35.45,10.994901048123777,35.55165622800387,11.09725485069693);
    out body;
    
[overpass] downloading data: [timeout:25][out:json];
        
    node["highway"="crossing"](35.45,10.9949010481237

['highway', 'crossing', 35.481036, 11.0534174, 2279789012]
['highway', 'crossing', 35.5106675, 11.0533385, 2497777676]
['highway', 'crossing', 35.5109902, 11.0527825, 2497777680]
['highway', 'crossing', 35.5109974, 11.0529285, 2497777695]
['highway', 'crossing', 35.5111874, 11.0524967, 2497777714]
['highway', 'crossing', 35.5112469, 11.0526324, 2497777742]
['highway', 'crossing', 35.5050862, 11.0628381, 6434026973]
['highway', 'crossing', 35.5441761, 11.0286931, 6434079323]
['highway', 'crossing', 35.5404338, 11.0294683, 6643390290]
['highway', 'crossing', 35.5403619, 11.0292025, 6643390291]
['highway', 'crossing', 35.5403545, 11.0294164, 6643390292]
['highway', 'crossing', 35.5414746, 11.0288873, 6643390293]
['highway', 'crossing', 35.5403269, 11.0292795, 6643390294]
['highway', 'crossing', 35.5414528, 11.0287537, 6643390295]
['highway', 'crossing', 35.5383136, 11.0302549, 6643390311]
['highway', 'crossing', 35.5382767, 11.0301254, 6643390312]
['highway', 'crossing', 35.5395491, 11.02

...

[overpass] downloading data: [timeout:25][out:json];
        
    node["tourism"="museum"](35.45,10.994901048123777,35.55165622800387,11.09725485069693);
    out body;
    


['tourism', 'artwork', 35.5098075, 11.0369729, 4728485089]
['tourism', 'artwork', 35.5438317, 11.0297511, 6643390376]


[overpass] downloading data: [timeout:25][out:json];
        
    node["direction"="anticlockwise"](35.45,10.994901048123777,35.55165622800387,11.09725485069693);
    out body;
    


['tourism', 'museum', 35.5037484, 11.0686118, 558552451]


[overpass] downloading data: [timeout:25][out:json];
        
    node["direction"="clockwise"](35.45,10.994901048123777,35.55165622800387,11.09725485069693);
    out body;
    
[overpass] downloading data: [timeout:25][out:json];
        
    node["substation"="minor distribution"](35.45,10.994901048123777,35.55165622800387,11.09725485069693);
    out body;
    
[overpass] downloading data: [timeout:25][out:json];
        
    node["route"="piste"](35.45,10.994901048123777,35.55165622800387,11.09725485069693);
    out body;
    
[overpass] downloading data: [timeout:25][out:json];
        
    node["route"="ferry"](35.45,10.994901048123777,35.55165622800387,11.09725485069693);
    out body;
    
[overpass] downloading data: [timeout:25][out:json];
        
    node["route"="canoe"](35.45,10.994901048123777,35.55165622800387,11.09725485069693);
    out body;
    
[overpass] downloading data: [timeout:25][out:json];
        
    node["route"="subway"](35.45,10.994901048123777,35.5516562

### Download parks

In [4]:
parks=[]
park_tags=[('leisure','park'),('landuse','grass'),('natural','wood'),('leisure','garden')]
####### parks
query="""
[out:json][timeout:25];
(
  way["leisure"="park"](42.189829808246266, -71.2912398492822, 42.48199986253678, -70.91697706096569);
  relation["leisure"="park"](42.189829808246266, -71.2912398492822, 42.48199986253678, -70.91697706096569);
  way["landuse"="grass"](42.189829808246266, -71.2912398492822, 42.48199986253678, -70.91697706096569);
  relation["landuse"="grass"](42.189829808246266, -71.2912398492822, 42.48199986253678, -70.91697706096569);
  way["natural"="wood"](42.189829808246266, -71.2912398492822, 42.48199986253678, -70.91697706096569);
  relation["natural"="wood"](42.189829808246266, -71.2912398492822, 42.48199986253678, -70.91697706096569);
);
// print results
out body;

    """
# result = overpass.query(query)

# for elt in result.elements():
#     feature=[key, value]
#     if elt.lat() and elt.lon():
#         feature.append(elt.lat())
#         feature.append(elt.lon())
#         feature.append(elt.id())
#         print(feature)
#         #print(divide_region(bbox, 4).index(region))
#         parks.append(feature)

for region in divide_region(bbox, 4):
    print(divide_region(bbox, 4).index(region))
    south=region[0]
    west=region[1]
    north=region[2]
    east=region[3]
    print(south, west, north, east)
    for tag in park_tags:
        query="""(
      way["""+tag[0]+"="+tag[1]+"""]("""+str(south)+','+str(west)+','+str(north)+','+str(east)+""");
      relation["""+tag[0]+"="+tag[1]+"""]("""+str(south)+','+str(west)+','+str(north)+','+str(east)+""");
    );
    out body;

        """
        print(query)
        result = overpass.query(query)

        for elt in result.elements():
            elt_info=(elt.id(),tag[0],tag[1],[(k.id(),k.lat(),k.lon()) for k in elt.nodes()])
            parks.append(elt_info)
#             feature=[key, value]
#             if elt.lat() and elt.lon():
#                 feature.append(elt.lat())
#                 feature.append(elt.lon())
#                 feature.append(elt.id())
#                 print(feature)
#                 print(divide_region(bbox, 4).index(region))
#                 parks.append(feature)



0
37.52092319 -122.55478906 37.62242953 -122.483693615
(
      way[leisure=park](37.52092319,-122.55478906,37.62242953,-122.483693615);
      relation[leisure=park](37.52092319,-122.55478906,37.62242953,-122.483693615);
    );
    out body;

        
(
      way[landuse=grass](37.52092319,-122.55478906,37.62242953,-122.483693615);
      relation[landuse=grass](37.52092319,-122.55478906,37.62242953,-122.483693615);
    );
    out body;

        
(
      way[natural=wood](37.52092319,-122.55478906,37.62242953,-122.483693615);
      relation[natural=wood](37.52092319,-122.55478906,37.62242953,-122.483693615);
    );
    out body;

        
(
      way[leisure=garden](37.52092319,-122.55478906,37.62242953,-122.483693615);
      relation[leisure=garden](37.52092319,-122.55478906,37.62242953,-122.483693615);
    );
    out body;

        
1
37.62242953 -122.55478906 37.72393587 -122.483693615
(
      way[leisure=park](37.62242953,-122.55478906,37.72393587,-122.483693615);
      relation[leis

In [7]:
### Adding what's left of Parks

for tag in park_tags:
    query="""(
  way["""+tag[0]+"="+tag[1]+"""]("""+str(bbox[0])+','+str(bbox[1])+','+str(bbox[2])+','+str(bbox[3])+""");
  relation["""+tag[0]+"="+tag[1]+"""]("""+str(bbox[0])+','+str(bbox[1])+','+str(bbox[2])+','+str(bbox[3])+""");
);
out body;

    """
    print(query)
    result = overpass.query(query)

    for elt in result.elements():
        elt_info=(elt.id(),tag[0],tag[1],[(k.id(),k.lat(),k.lon()) for k in elt.nodes()])
        parks.append(elt_info)



(
  way[leisure=park](37.52092319,-122.55478906,37.92694855,-122.27040728);
  relation[leisure=park](37.52092319,-122.55478906,37.92694855,-122.27040728);
);
out body;

    
(
  way[landuse=grass](37.52092319,-122.55478906,37.92694855,-122.27040728);
  relation[landuse=grass](37.52092319,-122.55478906,37.92694855,-122.27040728);
);
out body;

    
(
  way[natural=wood](37.52092319,-122.55478906,37.92694855,-122.27040728);
  relation[natural=wood](37.52092319,-122.55478906,37.92694855,-122.27040728);
);
out body;

    
(
  way[leisure=garden](37.52092319,-122.55478906,37.92694855,-122.27040728);
  relation[leisure=garden](37.52092319,-122.55478906,37.92694855,-122.27040728);
);
out body;

    


In [8]:
import pandas as pd
park_df=pd.DataFrame(parks, columns=['id','key','value','nodes'])

In [10]:
park_df.drop_duplicates(subset='id',inplace=True)

In [12]:
park_df.to_csv('parks_SF.csv',index=False)

In [13]:
elt.id()

7533494

In [14]:
result.elements()[0].nodes()[0].lat()

37.7624394

In [38]:
# map_features=[]
# for key in list(tag_dict.keys()):
#     for value in tag_dict[key]:
#         query="""

# area[name="Boston"]->.searchArea;
# (
#   node["""+'"'+key+'"'+"""="""+'"'+value+'"'+"""](area.searchArea);
#   way["""+'"'+key+'"'+"""="""+'"'+value+'"'+"""](area.searchArea);
#   relation["""+'"'+key+'"'+"""="""+'"'+value+'"'+"""](area.searchArea);
# );
# out body;
#     """
#         result = overpass.query(query)
        
#         for elt in result.elements():
#             feature=[key, value]
#             if elt.lat() and elt.lon():
#                 feature.append(elt.lat())
#                 feature.append(elt.lon())
#                 feature.append(elt.id())
#                 map_features.append(feature)

# 3) Outputting to CSV

In [37]:
import pandas as pd

In [38]:
map_features_df=pd.DataFrame(map_features, columns=["key","value","latitude","longitude","id"])
map_features_df.set_index("id", inplace=True)

In [43]:
map_features_df['id']=map_features_df.index

In [44]:
map_features_df.drop_duplicates(subset='id',inplace=True)

In [46]:
# map_features_df.to_csv("map_features_full_area.csv")